# ENHANCED RC4

In [1]:
#LIBRERIAS
from random import randint
import time

#FUNCIONES

#Given a 5 bytes/5 numbers(0-255) (40 bits input)
#generate a secret key of 32 bits (4 bytes/4 numbers(0-255))
def generateSecretKey(array):
    
    secretKey = []
    
    if len(array) == 5:
        
        bits = ""
        secretKeyStr = ""
        
        #Concatenar los 40 bits
        for i in array:
            #byte representation of each number
            strbyte = format(i,'#010b')
            
            #Remove the first 2 elements of strbyte that
            #do not form part of the byte
            bits = bits + strbyte[2:]
            
            #Remove the 1st bit of each byte to get
            #35 bits
            secretKeyStr = secretKeyStr + strbyte[3:]
            
        #End for 1  
        
        #Remove First, last and middle bit of 35 to get 32 bits
        secretKeyStr = secretKeyStr[1:16]+secretKeyStr[17:34]
        
        print("40 bits input:")
        print(bits)
        print("length:", len(bits) )
        
        print("32 bits input:")
        print(secretKeyStr)
        print("length:", len(secretKeyStr) )
        
        #Convert array of bits to integers
        for i in range(4):
            
            bytestr = secretKeyStr[(i*8):(i*8+8)]
            integer = int( bytestr,2 )
            secretKey.append(integer)
            #------------------
            print(i,") ",bytestr)
            print(integer)
            #------------------
            
        #End for 2
    
    #end if
    else:
        print("Error, input debe ser de 5 bytes/5 numeros 0-255")
    #Ene else    
    
    return secretKey
#-----------------------------
#Generate unique IV from n number of random
#integers [0-maxRandNumber]
def generateIV(sizeIV,maxRandNumber):
    
    iv = []
    
    for i in range(sizeIV):
        
        iv.append( randint(0,maxRandNumber) )
        
    #End for    
    
    return iv
    
#End function generateIV
#---------------------------
#Using the heap algorithm obtain all the 
#posible permiutations of agiven array
permutaciones = []

def permutations(lengLista ,lista):
    
    #Caso de salida
    if lengLista == 1:
        #print(lista)
        #Agregar permutaciones a variable global
        permutaciones.append(lista.copy())
    #End if 1   
    else :
        
        #Recursive call with unaltered lista
        permutations( (lengLista-1),lista )
        
        for i in range(lengLista-1):
            
            #Si es par
            if lengLista%2 == 0:
                swap( lista,i,(lengLista-1) )
            #End if 2
            #Si es impar
            else:
                swap( lista,0,(lengLista-1))
            #End else 2 
            
            #Recursive call with altered lista
            permutations( (lengLista-1),lista )
            
        #End for 1    
        
    #End else 1      
    
#End perutation function    
#---------------------------
def ksa(key1,key2,n):
    
    sone = []
    stwo = []
    
    #Because i in range only iterates until 1 number before the
    #max number, in both fors, we add 1 in n/2-1 and n-1 in order
    #to reach all the numbers
    for i in range( round(n/2) ):
        sone.append(i)
    #End for 1    
    
    for i in range( round(n/2) , (n) ):
        stwo.append(i)
    #End for 2    
    
    #-----------
    #print("S-box one:")
    #print(sone)
    #print("S-box two:")
    #print(stwo)
    #----------
    
    j = 0
    
    #Iteration for each s-box
    
    #First s-box
    for i in range(round(n/2)):
        
        actualValKey = key1[ mod(i, len(key1) )]
        actualTotalVal = j + sone[i] +actualValKey
        j = ( mod(actualTotalVal, round(n/2) ) )
        #---------
        #print("i: ",i,"j: ",j,"array length:",len(sone))
        #---------
        swap(sone,i,j)
        
    #End for
    
    #Second s-box
    for i in range(round(n/2)):
        
        actualValKey = key2[ mod(i, len(key2) )]
        actualTotalVal = j + stwo[i] +actualValKey
        j = ( mod(actualTotalVal, round(n/2) ) )
        #---------
        #print("i: ",i,"j: ",j,"array length:",len(stwo))
        #---------
        swap(stwo,i,j)
        
    #End for
    
    #print("-- S-boxes after Key Scheduling Algorithm --")
    #print("S-box one:")
    #print(sone)
    #print("S-box two:")
    #print(stwo)
    
    return[sone,stwo]
    
#End function ksa  
#----------------------------------------
def swapDifferentArrays(arrayi,posi,arrayj,posj):
    
    tempVar = arrayi[posi]
    arrayi[posi] = arrayj[posj]
    arrayj[posj] = tempVar 
    
    return [arrayi, arrayj]
#End function swap
#----------------------------------------
def swap(array,posi,posj):
    
    tempVar = array[posi]
    array[posi] = array[posj]
    array[posj] = tempVar 
    
    return array
#End function swap
#----------------------------------------
def mod(number,mod):
    return (number%mod)
#End function
#----------------------------------------
#Given to decilmal numbers perfrom XOR 
#and return value in decimal form
def xorDec(none,ntwo):
    return (none^ntwo)
#End fucntion xorDec
#----------------------------------------
#Convert array of integers to it is character
#representation
def intArrToStr(integers):
    
    string = []
    
    for i in integers:
        string.append(chr(i))
    #End for     
    
    return string
    
#End fufnction    
#----------------------------------------
def pgra(sone,stwo,plainText,n):
    
    i = 0
    j1 = 0
    j2 = 0
    keysSequence = []
    cipherText = []
    
    #For each letter of the plain text
    numberChars = len(plainText)
    
    #-------
    #print("Sbox1 length:",len(sone))
    #print("Sbox2 length:",len(stwo))
    #print("Plain text length:", len(plainText))
    #-------
    
    while i < numberChars:
        
        i = mod( (i+1), round(n/2))
        
        j1 = mod( (j1+sone[i+1]) , round(n/2)) 
        newArrays = swapDifferentArrays(sone,(i+1),stwo,(j1+1))
        sone = newArrays[0]
        stwo = newArrays[1]
        
        indx1 = mod( (sone[i+1]+sone[j1+1]), round(n/2) )
        temp1 = sone[indx1]
        t1 = temp1 + 1
        
        j2 = mod( (j2+stwo[i+1]) , round(n/2))
        newArrays = swapDifferentArrays(stwo,(i+1),sone,(j2+1))
        sone = newArrays[0]
        stwo = newArrays[1]
        
        indx2 = mod( (stwo[i+1]+stwo[j2+1]), round(n/2) )
        temp2 = stwo[indx2]
        t2 = temp2 + 1
        
        #Add the result of t1 XOR t2 to the key sequence
        keysSequence.append(xorDec(t1,t2))
        
        #-------
        #print("------")
        #print("i: ",i)
        #print("j1: ",j1)
        #print("t1: ",t1)
        #print("j2: ",j2)
        #print("t2: ",t2)
        #------- 
        
    #End while  
    
    #print("Keys for each character of plain text:")
    #print(keysSequence)
        
    #Xor between key sequence and the plain text
    for i in range( len(keysSequence) ):
        cipherText.append( xorDec( keysSequence[i], ord(plainText[i]) ) )
    #End for   
    
    return cipherText
    
#End function pgra
#----------------------------------------

<h2 style='background-color:cyan';>KEYS GENERATION</h2>

<h2 style='background-color:yellow';>Select secret key from sub-keys set</h2>

In [2]:
secretKey = generateSecretKey([27,89,90,12,155])
print("Secret key:")
print(secretKey)

40 bits input:
0001101101011001010110100000110010011011
length: 40
32 bits input:
01101110110011010100001100001101
length: 32
0 )  01101110
110
1 )  11001101
205
2 )  01000011
67
3 )  00001101
13
Secret key:
[110, 205, 67, 13]


In [3]:
permutations(len(secretKey),secretKey)
print("Sub-keys set:")
print(permutaciones)
print("#Permutaciones '!arrayLength' :", len(permutaciones) ) 

Sub-keys set:
[[110, 205, 67, 13], [205, 110, 67, 13], [67, 110, 205, 13], [110, 67, 205, 13], [205, 67, 110, 13], [67, 205, 110, 13], [13, 205, 110, 67], [205, 13, 110, 67], [110, 13, 205, 67], [13, 110, 205, 67], [205, 110, 13, 67], [110, 205, 13, 67], [110, 67, 13, 205], [67, 110, 13, 205], [13, 110, 67, 205], [110, 13, 67, 205], [67, 13, 110, 205], [13, 67, 110, 205], [13, 67, 205, 110], [67, 13, 205, 110], [205, 13, 67, 110], [13, 205, 67, 110], [67, 205, 13, 110], [205, 67, 13, 110]]
#Permutaciones '!arrayLength' : 24


In [4]:
#Select a random secret key from the set of Sub-keys set
maxRandNumber = len(permutaciones)-1
secKey = permutaciones[ randint(0,maxRandNumber) ]

<h2 style='background-color:yellow';>Generate unique Initial Vector</h2>

In [5]:
#Intital vector of 4 random integers [0-255] = Initital Vector of 32 bits
iv = generateIV(4,255)

print("Session key = Secret key (32 bits) + IV (32 bits)")
print("Secret key: ", secKey)
print("IV: ", iv)

Session key = Secret key (32 bits) + IV (32 bits)
Secret key:  [205, 13, 110, 67]
IV:  [74, 143, 21, 103]


<h2 style='background-color:cyan';>RC4 ENCRYPTION</h2>

<h2 style='background-color:yellow';>Key Scheduling Algorithm</h2>

In [6]:
initialTime = time.clock()
sboxes = ksa(secKey,iv,256)
#Pass arrays by value, since this are modified by the PRGA
newSone = sboxes[0].copy()
newStwo = sboxes[1].copy()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


<h2 style='background-color:yellow';>Pseudo Random Generation Algorithm</h2>

In [7]:
cipherTextDecimal = pgra(newSone,newStwo,["E","S","O"," ","E","S"," ","T","O","D","O"," ","A","M","I","G","O","S"],256)
finalTime = time.clock()
print("cipher text:")
print(cipherTextDecimal)

cipherText = intArrToStr(cipherTextDecimal)
print(cipherText)
print("Time: ", finalTime-initialTime)

cipher text:
[24, 114, 89, 14, 103, 73, 124, 225, 75, 23, 200, 76, 206, 118, 214, 166, 109, 117]
['\x18', 'r', 'Y', '\x0e', 'g', 'I', '|', 'á', 'K', '\x17', 'È', 'L', 'Î', 'v', 'Ö', '¦', 'm', 'u']
Time:  0.01999899999999999


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


<h2 style='background-color:cyan';>RC4 DECRYPTION</h2>

In [8]:
#Reset sboxes with values generated by KSA:
#Pass arrays by value, since this are modified by the PRGA
newSone = sboxes[0].copy()
newStwo = sboxes[1].copy()

#Perform XOR between the keysequence, generated from sone and stwo, and the cipherText
plainTextDecimal = pgra(newSone,newStwo, cipherText,256)
print("plain text:")
print(plainTextDecimal)

plaint = intArrToStr(plainTextDecimal)
print(plaint)

plain text:
[69, 83, 79, 32, 69, 83, 32, 84, 79, 68, 79, 32, 65, 77, 73, 71, 79, 83]
['E', 'S', 'O', ' ', 'E', 'S', ' ', 'T', 'O', 'D', 'O', ' ', 'A', 'M', 'I', 'G', 'O', 'S']
